In [12]:
import quantecon
import numpy as np 
import matplotlib.pyplot as plt
import copy

In [23]:
help(LinearStateSpace)

Help on class LinearStateSpace in module quantecon._lss:

class LinearStateSpace(builtins.object)
 |  LinearStateSpace(A, C, G, H=None, mu_0=None, Sigma_0=None)
 |  
 |  A class that describes a Gaussian linear state space model of the
 |  form:
 |  
 |  .. math::
 |  
 |    x_{t+1} = A x_t + C w_{t+1}
 |  
 |    y_t = G x_t + H v_t
 |  
 |  where :math:`{w_t}` and :math:`{v_t}` are independent and standard normal
 |  with dimensions k and l respectively.  The initial conditions are
 |  :math:`\mu_0` and :math:`\Sigma_0` for :math:`x_0 \sim N(\mu_0, \Sigma_0)`.
 |  When :math:`\Sigma_0=0`, the draw of :math:`x_0` is exactly :math:`\mu_0`.
 |  
 |  Parameters
 |  ----------
 |  A : array_like or scalar(float)
 |      Part of the state transition equation.  It should be `n x n`
 |  C : array_like or scalar(float)
 |      Part of the state transition equation.  It should be `n x m`
 |  G : array_like or scalar(float)
 |      Part of the observation equation.  It should be `k x n`
 |  H : 

In [5]:
class Person():
    def __init__(self,a,b):
        self.a = a
        self.b = b
    def get_utility(self,mu,var):
        return (self.a * mu - 0.5 * self.b * (var))

class Profession():
    def __init__(self,T,k,income_process):
        self.T = T
        self.k = k
        self.income_process = income_process

class Career():
    def __init__(self,A,C,G,mu0,sigma0,beta,T,k,a,b):
        if ((k<0) or (k>=T)):
            raise ValueError("k must be in the range [0, T-1]")
        
        if ((beta < 0) or (beta>1)):
            raise ValueError("beta must be in the range [0, 1]")
        
        self.A = A
        self.C = C
        self.G = G
        self.mu0 = mu0
        self.sigma0 = sigma0
        self.beta = beta 
        self.T = T
        self.k = k
        self.a = a
        self.b = b

        
    
    def construct_income_process(self):
        lss = quantecon.LinearStateSpace(A = self.A, C= self.C, G= self.G,
                                         mu_0=self.mu0,Sigma_0=(self.sigma0)**2)
        return lss
    
    def construct_profession(self,income_process):
        profession = Profession(self.T,self.k,income_process)
        return profession
    
    def get_unconditional_mean_PV0(self,profession):
        T = profession.T
        k = profession.k
        income_process = copy.deepcopy(profession.income_process)

        m = income_process.moment_sequence()
        discounted_mean = []

        for t in range(T):
            μ_x, μ_y, Σ_x, Σ_y = next(m)
            discounted_mean.append(float(μ_y) * (self.beta ** t))
        
        exp_PV0 = sum(discounted_mean[k:])
        return exp_PV0
    
    def get_unconditional_var_PV0(self,profession):
        T = profession.T
        k = profession.k
        income_process = copy.deepcopy(profession.income_process)

        m = income_process.moment_sequence()
        discounted_var = []
        for t in range(T):
            μ_x, μ_y, Σ_x, Σ_y= next(m)
            discounted_var.append(float(Σ_y) * (self.beta ** (2*t)))
        
        exp_var0 = sum(discounted_var[k:])
        return exp_var0
    
    def get_conditional_mean_PV0(self,profession):
        T = profession.T
        k = profession.k
        income_process = copy.deepcopy(profession.income_process)
        income_process.Sigma_0=np.zeros(income_process.Sigma_0.shape)  # set x0 to deterministic 

        m = income_process.moment_sequence()
        discounted_mean = []

        for t in range(T):
            μ_x, μ_y, Σ_x, Σ_y = next(m)
            discounted_mean.append(float(μ_y) * (self.beta ** (t)))
        
        exp_PV0 = sum(discounted_mean[k:])
        return exp_PV0
    
    def get_conditional_var_PV0(self,profession):
        T = profession.T
        k = profession.k
        income_process = copy.deepcopy(profession.income_process)
        income_process.Sigma_0=np.zeros(income_process.Sigma_0.shape)  # set x0 to deterministic 

        m = income_process.moment_sequence()
        discounted_var = []
        for t in range(T):
            μ_x, μ_y, Σ_x, Σ_y= next(m)
            discounted_var.append(float(Σ_y) * (self.beta ** (2*t)))
        
        exp_var0 = sum(discounted_var[k:])
        return exp_var0

    def construct_person(self):
    
        person = Person(self.a,self.b)
        return person
    
    def get_unconditional_EU(self,person,profession):
        mu = self.get_unconditional_mean_PV0(profession)
        var = self.get_unconditional_var_PV0(profession)

        EU = person.get_utility(mu,var)
        return EU
        

    def get_conditional_EU(self,person,profession):
        mu = self.get_conditional_mean_PV0(profession)
        var = self.get_conditional_var_PV0(profession)

        EU = person.get_utility(mu,var)
        return EU
        

In [56]:
#Profession-1 -> Doctor
A1 = np.array([[0.8, 0.2],
               [0.15, 0.85]])
C1 = np.eye(2)
G1 = np.array([[1.0, 0.0]])
mu_01 = np.array([1.5, 0.5])
Sigma_01 = np.eye(2) #There's no assumed correlation between the income fluctuations in each state.
k=6
T= 30



# Profession 2 -> Data Scientist
A2 = np.array([[0.8, 0.15, 0.05],  # Data Scientist
               [0.2, 0.6, 0.2],  # Mid-Level Data Scientist
               [0.1, 0.3, 0.6]])  # Junior Data Scientist/Intern
C2 = np.eye(3)
G2 = np.array([[1.0,0.8,0.6]])
mu_02 = np.array([1.8, 1.4, 1.0])
Sigma_02 = np.eye(3)
k2 = 5 

# Profession 3-> Designer
A3 = np.array([[0.75, 0.15, 0.1],  # Senior Designer
              [0.2, 0.6, 0.2],  # Mid-Level Designer
              [0.1, 0.3, 0.6]])  # Junior Designer/Intern
#C3 = np.eye(3)
C3 = np.array([[1.0, 0.2, 0.1],  # Senior Designer (experience, portfolio, certifications)
              [0.3, 0.6, 0.1],  # Mid-Level Designer (education, skills, experience)
              [0.1, 0.6, 0.3]])  # Junior Designer/Intern (education, portfolio)
G3 = np.array([1.2,1,0.8])
mu_03 = np.array([1.3, 1.0, 0.75])
Sigma_03 = np.eye(3)
k3 = 4

# **Example**- *Person with a=1, b=0.2*

**Profession 1- Doctor**

In [53]:
test1 = Career(A=A1,C=C1,G=G1,mu0=mu_01,sigma0=Sigma_01,beta=beta,T=T,k=k1,a=1,b=0.2)
income1 = test1.construct_income_process()
prof1 = test1.construct_profession(income1)
person1 = test1.construct_person()

print("unconditional mean pv0: ",test1.get_unconditional_mean_PV0(prof1))
print("unconditional var pv0: ",test1.get_unconditional_var_PV0(prof1))

print("conditional mean pv0: ",test1.get_conditional_mean_PV0(prof1))
print("conditional var pv0: ",test1.get_conditional_var_PV0(prof1))

print("uncondtional EU: " ,test1.get_unconditional_EU(person1,prof1))
print("condtional EU: " ,test1.get_conditional_EU(person1,prof1))


unconditional mean pv0:  8.36609692083572
unconditional var pv0:  35.03691787854541
conditional mean pv0:  8.36609692083572
conditional var pv0:  32.97993979960842
uncondtional EU:  4.862405132981179
condtional EU:  5.068102940874878


**Profession 2- Data Scientist**

In [54]:
test2 = Career(A=A2,C=C2,G=G2,mu0=mu_02,sigma0=Sigma_02,beta=beta,T=T,k=k2,a=1,b=0.2)
income2 = test2.construct_income_process()
prof2 = test2.construct_profession(income1)
person1 = test2.construct_person()

print("unconditional mean pv0: ",test2.get_unconditional_mean_PV0(prof2))
print("unconditional var pv0: ",test2.get_unconditional_var_PV0(prof2))

print("conditional mean pv0: ",test2.get_conditional_mean_PV0(prof2))
print("conditional var pv0: ",test2.get_conditional_var_PV0(prof2))

print("uncondtional EU: " ,test2.get_unconditional_EU(person1,prof2))
print("condtional EU: " ,test2.get_conditional_EU(person1,prof2))


unconditional mean pv0:  10.51819488038548
unconditional var pv0:  41.90572142717099
conditional mean pv0:  10.51819488038548
conditional var pv0:  39.0326199056804
uncondtional EU:  6.3276227376683805
condtional EU:  6.614932889817439


**Profession 3- Designer**

In [57]:
test3 = Career(A=A3,C=C3,G=G3,mu0=mu_03,sigma0=Sigma_03,beta=beta,T=T,k=k3,a=1,b=0.2)
income3 = test3.construct_income_process()
prof3 = test3.construct_profession(income1)
person1 = test3.construct_person()

print("unconditional mean pv0: ",test3.get_unconditional_mean_PV0(prof3))
print("unconditional var pv0: ",test3.get_unconditional_var_PV0(prof3))

print("conditional mean pv0: ",test3.get_conditional_mean_PV0(prof3))
print("conditional var pv0: ",test3.get_conditional_var_PV0(prof3))

print("uncondtional EU: " ,test3.get_unconditional_EU(person1,prof3))
print("condtional EU: " ,test3.get_conditional_EU(person1,prof3))

unconditional mean pv0:  11.35760465897923
unconditional var pv0:  44.06468400948721
conditional mean pv0:  11.35760465897923
conditional var pv0:  40.85863190087925
uncondtional EU:  6.95113625803051
condtional EU:  7.271741468891305


# **Example**- *Person with a=1, b=0.65*

**Profession 1- Doctor**

In [70]:
test1 = Career(A=A1,C=C1,G=G1,mu0=mu_01,sigma0=Sigma_01,beta=beta,T=T,k=k1,a=1,b=0.65)
income1 = test1.construct_income_process()
prof1 = test1.construct_profession(income1)
person2 = test1.construct_person()

print("unconditional mean pv0: ",test1.get_unconditional_mean_PV0(prof1))
print("unconditional var pv0: ",test1.get_unconditional_var_PV0(prof1))

print("conditional mean pv0: ",test1.get_conditional_mean_PV0(prof1))
print("conditional var pv0: ",test1.get_conditional_var_PV0(prof1))

print("uncondtional EU: " ,test1.get_unconditional_EU(person2,prof1))
print("condtional EU: " ,test1.get_conditional_EU(person2,prof1))


unconditional mean pv0:  8.36609692083572
unconditional var pv0:  35.03691787854541
conditional mean pv0:  8.36609692083572
conditional var pv0:  32.97993979960842
uncondtional EU:  -3.0209013896915398
condtional EU:  -2.352383514037017


**Profession 2- Data Scientist**

In [69]:
test2 = Career(A=A2,C=C2,G=G2,mu0=mu_02,sigma0=Sigma_02,beta=beta,T=T,k=k2,a=1,b=0.65)
income2 = test2.construct_income_process()
prof2 = test2.construct_profession(income1)
person2 = test2.construct_person()

print("unconditional mean pv0: ",test2.get_unconditional_mean_PV0(prof2))
print("unconditional var pv0: ",test2.get_unconditional_var_PV0(prof2))

print("conditional mean pv0: ",test2.get_conditional_mean_PV0(prof2))
print("conditional var pv0: ",test2.get_conditional_var_PV0(prof2))

print("uncondtional EU: " ,test2.get_unconditional_EU(person2,prof2))
print("condtional EU: " ,test2.get_conditional_EU(person2,prof2))

unconditional mean pv0:  10.51819488038548
unconditional var pv0:  41.90572142717099
conditional mean pv0:  10.51819488038548
conditional var pv0:  39.0326199056804
uncondtional EU:  -3.101164583445092
condtional EU:  -2.1674065889606506


**Profession 3- Designer**

In [68]:
test3 = Career(A=A3,C=C3,G=G3,mu0=mu_03,sigma0=Sigma_03,beta=beta,T=T,k=k3,a=1,b=0.65)
income3 = test3.construct_income_process()
prof3 = test3.construct_profession(income1)
person2 = test3.construct_person()

print("unconditional mean pv0: ",test3.get_unconditional_mean_PV0(prof3))
print("unconditional var pv0: ",test3.get_unconditional_var_PV0(prof3))

print("conditional mean pv0: ",test3.get_conditional_mean_PV0(prof3))
print("conditional var pv0: ",test3.get_conditional_var_PV0(prof3))

print("uncondtional EU: " ,test3.get_unconditional_EU(person2,prof3))
print("condtional EU: " ,test3.get_conditional_EU(person2,prof3))

unconditional mean pv0:  11.35760465897923
unconditional var pv0:  44.06468400948721
conditional mean pv0:  11.35760465897923
conditional var pv0:  40.85863190087925
uncondtional EU:  -2.963417644104112
condtional EU:  -1.9214507088065265


The benefit of having separate classes for Profession and Person is that we can represent the Profession and Person individually for the discussion of their own characteristics

A profession can be 'uniquely' defined by its max career length `T`, education requirement `k`, and income stream (using `LinearStateSpace`). 

Similarly, a person can be described by his or her risk appetite `a` and `b`